In [1]:
import numpy as np
import pennylane as qml
from math import log2, ceil

##############################################################################
# 1. Bosonic q, p in truncated HO basis
##############################################################################

def create_matrix(cutoff, kind, m=1.0):
    """Return q or p in HO Fock basis up to 'cutoff'."""
    mat = np.zeros((cutoff, cutoff), dtype=np.complex128)
    for i in range(cutoff):
        if i > 0:
            if kind == "q":
                mat[i, i-1] = (1.0/np.sqrt(2.0*m)) * np.sqrt(i)
            elif kind == "p":
                mat[i, i-1] = 1j*np.sqrt(m/2.0) * np.sqrt(i)
        if i < cutoff - 1:
            if kind == "q":
                mat[i, i+1] = (1.0/np.sqrt(2.0*m)) * np.sqrt(i+1)
            elif kind == "p":
                mat[i, i+1] = -1j*np.sqrt(m/2.0) * np.sqrt(i+1)
    return mat

##############################################################################
# 2. Dense bosonic blocks only (size cutoff x cutoff)
##############################################################################

def bosonic_blocks(cutoff, potential, m=1.0, g=1.0, u=1.0):
    """
    Build bosonic dense pieces:
      A_b = p^2 + W'(q)^2
      B_b = W''(q)
    in the boson Fock basis.
    """
    q = create_matrix(cutoff, "q", m=m)
    p = create_matrix(cutoff, "p", m=m)

    q2 = q @ q
    q3 = q2 @ q
    I_b = np.eye(cutoff, dtype=np.complex128)

    if potential == "QHO":
        W_prime = m * q
        W_pp    = m * I_b
    elif potential == "AHO":
        W_prime = m * q + g * q3
        W_pp    = m * I_b + 3.0 * g * q2
    elif potential == "DW":
        W_prime = m * q + g * q2 + g * (u**2) * I_b
        W_pp    = m * I_b + 2.0 * g * q
    else:
        raise ValueError("potential must be 'QHO', 'AHO', or 'DW'.")

    p2 = p @ p
    A_b = p2 + (W_prime @ W_prime)
    B_b = W_pp

    # clean tiny numerical noise
    A_b[np.abs(A_b) < 1e-12] = 0.0
    B_b[np.abs(B_b) < 1e-12] = 0.0
    return A_b, B_b

##############################################################################
# 3. Boson dense -> Pauli on boson qubits (padded)
##############################################################################

def embed_dense_to_qubits(H_dense):
    """
    Embed (d x d) dense block into (2^n x 2^n) by padding top-left.
    Returns padded matrix and n qubits.
    """
    d = H_dense.shape[0]
    n_qubits = int(ceil(log2(d)))
    dim_q = 2**n_qubits
    H_q = np.zeros((dim_q, dim_q), dtype=np.complex128)
    H_q[:d, :d] = H_dense
    return H_q, n_qubits

def pauli_decompose_dense(H_dense):
    """Dense boson block -> padded -> qml.Hamiltonian on boson wires 0..n_b-1."""
    H_q, n_b = embed_dense_to_qubits(H_dense)
    H_pl = qml.pauli_decompose(H_q)
    return H_pl, n_b

def strip_zero_terms(H, tol=1e-12):
    coeffs, ops = [], []
    for c, o in zip(H.coeffs, H.ops):
        if abs(c) > tol:
            coeffs.append(c)
            ops.append(o)
    return qml.Hamiltonian(coeffs, ops)

##############################################################################
# 4. Full SUSY QM Hamiltonian assembled in Pauli form (no full matrix)
##############################################################################

def build_susy_qm_hamiltonian(
    cutoff,
    potential,
    m=1.0,
    g=1.0,
    u=1.0,
    remove_zero_terms=True,
    fermion_wire=0
):
    """
    Build SUSY QM Hamiltonian as a qml.Hamiltonian without forming the full
    fermion⊗boson dense matrix.

    Wires:
      - fermion qubit on fermion_wire (default 0)
      - boson qubits on fermion_wire+1, ..., fermion_wire+n_b
    """
    # Bosonic dense blocks only
    A_b, B_b = bosonic_blocks(cutoff, potential, m=m, g=g, u=u)

    # Pauli-decompose boson blocks (on local boson wires 0..n_b-1)
    H_A_pl, n_b = pauli_decompose_dense(A_b)
    H_B_pl, _   = pauli_decompose_dense(B_b)

    # Shift boson wires -> global boson wires (fermion wire first)
    boson_wire_map = {w: fermion_wire + 1 + w for w in range(n_b)}

    full_coeffs = []
    full_ops = []

    # 1/2 * I_f ⊗ A_b
    for c, op in zip(H_A_pl.coeffs, H_A_pl.ops):
        op_b = qml.map_wires(op, boson_wire_map)
        full_ops.append(qml.prod(qml.Identity(fermion_wire), op_b))
        full_coeffs.append(0.5 * c)

    # 1/2 * Z ⊗ B_b
    for c, op in zip(H_B_pl.coeffs, H_B_pl.ops):
        op_b = qml.map_wires(op, boson_wire_map)
        full_ops.append(qml.prod(qml.PauliZ(fermion_wire), op_b))
        full_coeffs.append(0.5 * c)

    H_total = qml.Hamiltonian(full_coeffs, full_ops).simplify()
    if remove_zero_terms:
        H_total = strip_zero_terms(H_total)

    n_total_qubits = 1 + n_b
    return H_total, n_total_qubits, n_b


In [ ]:
H_pl, n_qubits, n_b = build_susy_qm_hamiltonian(
    cutoff=1024,
    potential="DW",
    m=1.0, g=1.0, u=1.0
)

print(n_qubits, "total qubits =", 1, "fermion +", n_b, "boson")
print("Number of Pauli terms:", len(H_pl.ops))


In [4]:
from susy_qm import calculate_Hamiltonian

potential="DW"
cutoff=1024
H = calculate_Hamiltonian(cutoff, potential)
H_pauli = qml.pauli_decompose(H)

len(H_pauli.ops)

23416